In [122]:
import numpy as np
from numba import jit, njit


In [123]:
def get_nq_and_eps(Nq_max, A, lamb, n1, n2):
    # Determine the parity of Nq - lamb
    Nqpar = Nq_max % 2
    Nq_minus_lamb_par = (Nqpar - lamb) % 2
    eps = Nq_minus_lamb_par % 2
    # Determine Nq
    Nq = 2 * n1 + 2 * n2 + 2 * A + lamb + eps

    return Nq, eps


def get_ells(A, B, lamb, eps):    
    l1 = A + B + eps
    l2 = A - B + lamb

    return l1, l2

# Based on the stupid order things are saved in FORTRAN (yes, makes no sense)
# From: N' (B'), n1', M' (A'), n1, n2, N (B), M (A), L (lamb)
# To: n1, l1, n2, l2, n1', l1', l2', lamb
def recast_indices(Nq_max, Bprime, n1prime, Aprime, n1, n2, B, A, lamb):

    Nq, eps = get_nq_and_eps(Nq_max, A, lamb, n1, n2)
    l1, l2 = get_ells(A, B, lamb, eps)
    l1prime, l2prime = get_ells(Aprime, Bprime, lamb, eps)

    return [n1, l1, n2, l2, n1prime, l1prime, l2prime, lamb]



In [125]:
# Prepare the field
lamb_max = 10
lamb_min = 0
nqdif = 2
Nq_max = nqdif + lamb_max

sz = (Nq_max - lamb_min)//2 + 1
brackets_f = np.zeros((lamb_max + 1, sz, sz, sz, sz, lamb_max + 1, sz, lamb_max - lamb_min + 1))

# Open the file and load the Fortran computed values
values_and_indices_f = np.loadtxt('OSBRACKETS/out_even')
values_f = values_and_indices_f[:, -1]
indices_f = values_and_indices_f[:, :-1].astype(int)

# Index the matrix
# Based on the stupid order things are saved in FORTRAN (yes, makes no sense)
# N' (B'), n1', M' (A'), n1, n2, N (B), M (A), L (lamb)
brackets_f[tuple(indices_f.T)] = values_f

# --------------------------------------------
# Brackets with the quantum numbers we want:
# n1, l1, n2, l2, n1', l1', l2', lamb   (  n2' = (2*n1 + l1 + 2*n2 + l2 - 2*n1' - l1'  - l2')/2  )
l_min = 0
l_max = Nq_max
n_max = (Nq_max - lamb_min)//2
brackets = np.zeros((n_max + 1, l_max - l_min + 1, n_max + 1, l_max - l_min + 1, n_max + 1, 
                                l_max - l_min + 1, l_max - l_min + 1, lamb_max - lamb_min + 1))

# Recast the indices (incredibly works)
indices = np.vstack(recast_indices(Nq_max, *[indices_f[:, i] for i in range(8)])).T
# Index the matrix
brackets[tuple(indices.T)] = values_f

# Some tests
print(indices_f[12])
print(recast_indices(Nq_max, *indices_f[12]))
print(indices[12])



for i in range(54000, len(values_f)):
    break
    idx = indices_f[i]

    n1 = idx[3]
    n2 = idx[4]
    lamb = idx[7]
    A, B = idx[6], idx[5]
    Nq, eps = get_nq_and_eps(Nq_max, A, lamb, n1, n2)
    l1, l2 = get_ells(A, B, lamb, eps)

    # if i % 1000 == 0:
    #     print(i)

    if values_f[i] != 0:
        
        #print(l1, l2, l1 + l2)

        Nq = 2 * n1 + l1 + 2 * n2 + l2
        if (Nq_max - Nq) % 2 != 0:
            print("SDADd")

        if Nq == 5:
            print(Nq_max - Nq, Nq, eps)
            print(lamb, l1, l2, n1, n2)
            print('---')
            # print(idx, values_f[i])

        # print(idx, values[i])
        # print(idx, brackets_f[tuple(idx.T)], "Idx")


KeyboardInterrupt: 